In [ ]:
from __future__ import annotations
from typing import Optional, Tuple, Iterable, List, Dict, Set, Any
import io
import os
import time
import math
import re
import pickle
import random
from collections import Counter

In [ ]:
"""
Implementação do modelo PPM (Prediction by Partial Matching) - C

Descrição geral:
- Este arquivo contém uma versão do PPM Modelo C adaptada para uso com uma
  biblioteca de codificação aritmética (aqui referida como `arithmeticcoding`).
- O PPM é um modelo de previsão de símbolos para compressão de dados baseado em
  contextos de tamanho limitado (ordem). O Modelo C usa uma trie onde cada
  nó representa um símbolo no contexto e cada nó guarda contagens de ocorrência.
- O código inclui:
  - Estruturas de nó e árvore (NoPPM_C, ArvorePPM_C)
  - Integração com codificação aritmética (entrada/saída de bits)
  - Tratamento de símbolos especiais: símbolo de escape e símbolo de fim
  - Cálculo de entropia empírica enquanto codifica/decodifica, de modo simétrico
  - Funções utilitárias para compactar/descompactar e comparar arquivos

Observações técnicas importantes (em alto nível):
- A versão usa um dicionário em cada nó para acesso O(1) aos filhos (em média),
  ao invés de listas ligadas; isso melhora desempenho em grandes alfabetos.
- Mantemos ponteiros "vine" (vine/link) para o nó correspondente ao contexto
  reduzido (usado para percorrer contextos de ordem decrescente). Essa é uma
  otimização típica em PPM para acelerar busca de contextos menores.
- Para interagir com a codificação aritmética, este código constrói tabelas de
  frequência (SimpleFrequencyTable) a cada nível de contexto. Há um limite de
  somatório (limite) para escalar contagens muito grandes e evitar overflow.
- O símbolo de escape (_SIMBOLO_ESCAPE) é usado quando um símbolo não é
  observado no contexto atual: codifica-se uma ocorrência de escape e desce-se
  para contexto menor.
- O símbolo de fim (_SIMBOLO_FIM) é codificado ao finalizar a compressão para
  sinalizar término durante a descompressão.
"""
# Tenta importar a biblioteca de codificação aritmética implementada por Nayuki. Se não existir, termina.
try:
    import arithmeticcoding
except ImportError:
    print("Erro: A biblioteca 'arithmetic-coding' não foi encontrada.")
    raise

# -------------------
# Símbolos especiais
# -------------------
# Usamos objetos únicos para representar ESCAPE e EOF. Não utilizamos
# valores inteiros fixos aqui para evitar colisões com símbolos válidos do
# alfabeto. Esses objetos são usados como chaves nos dicionários.
_SIMBOLO_ESCAPE = object()
_SIMBOLO_FIM = object()

# ---------------------------------
# Definição do nó da trie (NoPPM_C)
# ---------------------------------
class NoPPM_C:
    """
    Nó da trie para modelo PPM.

    Atributos principais:
    - simbolo: o símbolo representado por este nó (None para raiz).
    - contagem: número de vezes que o símbolo foi observado no contexto que o
      nó representa.
    - filhos: dicionário mapeando simbolo -> NoPPM_C (acesso O(1) médio).
    - vine: ponteiro para o próximo nó no encadeamento de contexto (equivale
      ao 'sufixo' / contexto reduzido). Usado para descer para contextos menores
      durante a codificação/decodificação.
    - pai: ponteiro para o nó pai (útil para navegação/atualizações).
    - nivel: profundidade do nó (0 para raiz, 1 para primeiro nível, ...)

    Observações de implementação:
    - __slots__ reduz o overhead de memória por nó, importante em tries
      grandes.
    - Ao criar filhos, inicializamos com contagem 1
    """

    __slots__ = ("simbolo", "contagem", "filhos", "vine", "pai", "nivel")

    def __init__(self, simbolo: Optional[Any], nivel: int = 0, pai: Optional["NoPPM_C"] = None):
        # simbolo: valor do símbolo (int para bytes, ou objetos especiais)
        self.simbolo: Optional[Any] = simbolo

        # contagem de ocorrências no contexto representado por este nó
        self.contagem: int = 1

        # filhos: dicionário para acesso rápido
        self.filhos: Dict[Any, NoPPM_C] = {}

        # vine: link para o nó que representa o contexto reduzido
        self.vine: Optional["NoPPM_C"] = None

        # pai: nó pai na trie
        self.pai: Optional["NoPPM_C"] = pai

        # nivel: profundidade (0 = raiz, a qual corresponde a coluna k = -1, seguindo a notação vista em sala)
        self.nivel: int = nivel

    def encontrar_filho(self, simbolo: Any) -> Optional["NoPPM_C"]:
        #Retorna o filho para o símbolo dado ou None se não existir.

        return self.filhos.get(simbolo)

    def obter_ou_adicionar_filho(self, simbolo: Any, contagem_inicial: int = 1) -> Tuple["NoPPM_C", bool]:
        """
        Retorna (filho, criado_flag).

        Se o filho existir, retorna o nó existente e False. Caso contrário,
        cria um novo nó com contagem inicial e retorna True no segundo elemento.
        """
        existente = self.encontrar_filho(simbolo)
        if existente:
            return existente, False

        novo_filho = NoPPM_C(simbolo=simbolo, nivel=self.nivel + 1, pai=self)
        novo_filho.contagem = contagem_inicial
        self.filhos[simbolo] = novo_filho

        return novo_filho, True

    def obter_ou_criar_no_escape(self) -> Tuple["NoPPM_C", bool]:
        #Conveniência para obter/criar o nó de escape no conjunto de filhos.
        return self.obter_ou_adicionar_filho(_SIMBOLO_ESCAPE, contagem_inicial=1)

    def __repr__(self) -> str:
        if self.simbolo is None:
            return "<RAIZ>"
        if self.simbolo is _SIMBOLO_ESCAPE:
            return f"<ESC,{self.contagem}>"
        return f"<{self.simbolo},{self.contagem}>"

# -----------------------------
# Arvore PPM (ArvorePPM_C)
# -----------------------------
class ArvorePPM_C:
    """
    Implementação do modelo PPM-C com integração à codificação
    aritmética.

    Campos principais:
    - ordem: ordem máxima do modelo (tamanho do contexto máximo em símbolos).
    - tamanho_alfabeto_bytes: tamanho do alfabeto básico (por exemplo 256 para bytes).
    - tamanho_alfabeto_total: alfabeto + símbolos especiais (escape e fim).
    - simbolo_para_int/int_para_simbolo: mapeamentos para tabela de frequências.
    - raiz/base/contexto: estruturas de navegação da trie.
    - soma_entropia/contagem_simbolos: para cálculo empírico da entropia H(X).

    Observações detalhadas:
    - A classe constrói tabelas de frequência a partir das contagens observadas
      em cada contexto e passa essas tabelas para o codificador/decodificador
      aritmético.
    - O método _criar_tabela_frequencia aplica uma escala/normalização quando
      a soma das contagens excede um limite (para evitar números muito grandes).
    - Os métodos codificar_simbolo e decodificar_simbolo mantêm a simetria do
      cálculo de entropia para comparação entre compressão e descompressão.
    """

    def __init__(self, ordem: int, tamanho_alfabeto: int = 256, fluxo_bits: io.BytesIO = None):
        if ordem < 0:
            raise ValueError("A ordem deve ser >= 0.")
        self.ordem: int = ordem
        self.tamanho_alfabeto_bytes = tamanho_alfabeto
        self.tamanho_alfabeto_total = tamanho_alfabeto + 2

        # Inicializa mapeamentos símbolo <-> inteiro (necessários para a tabela de frequência)
        self.simbolo_para_int: Dict[Any, int] = {i: i for i in range(tamanho_alfabeto)}
        self.int_para_simbolo: Dict[int, Any] = {i: i for i in range(tamanho_alfabeto)}

        esc_int = tamanho_alfabeto
        fim_int = tamanho_alfabeto + 1

        self.simbolo_para_int[_SIMBOLO_ESCAPE] = esc_int
        self.int_para_simbolo[esc_int] = _SIMBOLO_ESCAPE
        self.simbolo_para_int[_SIMBOLO_FIM] = fim_int
        self.int_para_simbolo[fim_int] = _SIMBOLO_FIM

        # Raiz/Base/Contexto para navegar na trie
        self.raiz: NoPPM_C = NoPPM_C(simbolo=None, nivel=0)
        self.base: NoPPM_C = self.raiz
        self.contexto: Tuple[int, ...] = tuple()

        # Para o cálculo empírico de entropia
        self.soma_entropia = 0.0
        self.contagem_simbolos = 0

        # Inicializa codificador/decodificador de acordo com o fluxo de bits
        if fluxo_bits:
            # Modo de decodificação (leitura de um fluxo já existente)
            self.leitor_bits = arithmeticcoding.BitInputStream(fluxo_bits)
            self.decodificador = arithmeticcoding.ArithmeticDecoder(32, self.leitor_bits)
        else:
            # Modo de codificação (escreve em um buffer interno)
            self.escritor_bits = arithmeticcoding.BitOutputStream(io.BytesIO())
            self.codificador = arithmeticcoding.ArithmeticEncoder(32, self.escritor_bits)

    # -----------------------------
    # Métodos auxiliares para probabilidades e tabelas
    # -----------------------------
    def _obter_probabilidades(self, no_contexto: NoPPM_C, simbolos_excluidos: Set[Any]) -> Optional[Dict[Any, int]]:
        """
        Retorna um dicionário símbolo -> contagem para os filhos visíveis do nó de
        contexto, ignorando símbolos que já foram excluídos (já vistos em
        contextos maiores ou processados como escapes).

        Retorna None se não houver filhos visíveis ou se a soma das contagens for 0.
        """
        filhos = no_contexto.filhos.values()
        visiveis = [c for c in filhos if c.simbolo not in simbolos_excluidos]

        if not visiveis:
            return None

        total = sum(c.contagem for c in visiveis)

        if total == 0:
            return None

        return {c.simbolo: c.contagem for c in visiveis}

    def _criar_tabela_frequencia(self, probs: Dict[Any, int]) -> arithmeticcoding.SimpleFrequencyTable:
        """
        Constrói uma SimpleFrequencyTable usada pela codificação aritmética a partir
        do dicionário de probabilidades (contagens) 'probs'.

        Cria uma lista de frequências do tamanho do alfabeto total e
        coloca a contagem correspondente na posição inteira do símbolo.

        Quando a soma total das contagens excede um limite predefinido (limite),
        aplicamos uma normalização/escala para evitar frequências muito grandes
        que possam causar overflow/ineficiências na implementação da aritmética.

        A escala é feita proporcionalmente; garantimos que qualquer símbolo com
        contagem > 0 tenha pelo menos frequência 1 após a escala (evita elementos
        arredondados para zero que quebrariam o modelo).
        """
        frequencias = [0] * self.tamanho_alfabeto_total
        total = sum(probs.values())
        limite = (1 << 14)  # Limite arbitrário alto para manter números pequenos

        if total > limite:
            for simbolo, contagem in probs.items():
                simbolo_int = self.simbolo_para_int[simbolo]
                contagem_escalada = (contagem * limite + total // 2) // total

                if contagem > 0 and contagem_escalada == 0:
                    # Protege contra arredondamento para 0
                    frequencias[simbolo_int] = 1
                else:
                    frequencias[simbolo_int] = contagem_escalada
        else:
            for simbolo, contagem in probs.items():
                simbolo_int = self.simbolo_para_int[simbolo]
                frequencias[simbolo_int] = contagem

        return arithmeticcoding.SimpleFrequencyTable(frequencias)

    # ---------------------------------------------------
    # Atualização do modelo (inserção/atualização de nós)
    # ---------------------------------------------------
    def _atualizar_modelo(self, simbolo: Any) -> None:
        """
        Atualiza a trie com a observação do símbolo dado, iniciando a partir do
        nó base (contexto atual) e subindo/descendo pela cadeia de vine conforme
        necessário.

        O algoritmo realiza o seguinte, em cada contexto desde o atual até None:
        - Tenta obter (ou criar) o filho correspondente ao símbolo
        - Liga o filho anterior atualizado pelo campo `vine` para manter
          encadeamentos entre os nós recém-criados (otimização de navegação)
        - Se o filho foi criado: garante existência do nó de escape no nó atual
          e segue para o vine do contexto (continua a atualizar em contextos
          menores)
        - Se o filho já existia: incrementa sua contagem e interrompe (pois a
          observação já está integrada no nó para contextos maiores)

        Finalmente atualiza o contexto corrente (self.contexto) e a base (self.base)
        para refletir o novo símbolo.

        Notas:
        - Tratamento de símbolos especiais: se o símbolo não for inteiro (por ex.
          _SIMBOLO_ESCAPE ou _SIMBOLO_FIM) reiniciamos o contexto para tuple()
          (isso evita usar símbolos especiais como parte do contexto para bytes).
        """
        no_contexto = self.base
        no_atualizado_anterior = None

        while no_contexto is not None:
            filho, criado = no_contexto.obter_ou_adicionar_filho(simbolo)

            if no_atualizado_anterior:
                # linka o nó atualizado anteriormente para o novo filho
                no_atualizado_anterior.vine = filho
            no_atualizado_anterior = filho

            if criado:
                # Se criamos um novo filho, garantimos que exista um nó de escape
                if no_contexto.encontrar_filho(_SIMBOLO_ESCAPE):
                    no_contexto.filhos[_SIMBOLO_ESCAPE].contagem += 1
                else:
                    no_contexto.obter_ou_criar_no_escape()
                # Desce para o vine do contexto (contexto menor)
                no_contexto = no_contexto.vine
            else:
                # Se o filho já existia, apenas incrementamos e paramos
                filho.contagem += 1
                break

        # Atualiza o contexto (desconsidera símbolos especiais)
        if isinstance(simbolo, int):
            self.contexto = (self.contexto + (simbolo,))[-self.ordem:]
        else:
            # Símbolos especiais não fazem parte do contexto
            self.contexto = tuple()

        # Recalcula self.base descendo a partir da raiz usando o novo contexto
        self.base = self.raiz

        for s in self.contexto:
            prox = self.base.encontrar_filho(s)
            if not prox:
                break
            self.base = prox

    # -----------------------------
    # Codificação de um símbolo (usando PPM com escapes)
    # -----------------------------
    def codificar_simbolo(self, simbolo: Any) -> None:
        """
        Codifica o símbolo passado usando o modelo PPM atual juntamente com o
        codificador aritmético.

        Algoritmo:
        - Começa no contexto atual (self.base) e tenta achar probabilidades
          condicionais (filhos visíveis)
        - Se o símbolo aparece nas probabilidades do contexto atual: codifica-o
          diretamente com a tabela e atualiza o modelo
        - Se não aparece, mas há símbolo de escape no contexto: codifica o
          símbolo de escape e desce para o vine (contexto menor), repetindo
        - Se chegamos a nenhum contexto com probabilidades (ordem -1), usamos
          a equiprobabilidade sobre os símbolos restantes (aqueles que não
          foram excluídos) e codificamos com essa tabela

        Além disso, calculamos a entropia empírica acumulando -log2(p) onde p é
        a probabilidade usada para representar o símbolo.
        """
        simbolo_int = self.simbolo_para_int[simbolo]
        excluidos = set()
        no_contexto = self.base
        while no_contexto is not None:
            probs = self._obter_probabilidades(no_contexto, excluidos)
            if probs and simbolo in probs:
                # Caso 1: símbolo encontrado no contexto atual
                tabela = self._criar_tabela_frequencia(probs)
                total = sum(probs.values())
                prob_simbolo = probs[simbolo] / total
                # Acumula entropia para análise (simetria entre codificar/decodificar)
                self.soma_entropia += -math.log2(prob_simbolo)
                self.contagem_simbolos += 1
                # Escreve o símbolo usando a tabela do contexto
                self.codificador.write(tabela, simbolo_int)
                # Atualiza o modelo com a observação
                self._atualizar_modelo(simbolo)
                return
            # Caso 2: símbolo não encontrado no contexto atual
            if probs and _SIMBOLO_ESCAPE in probs:
                #Codifica o símbolo de escape (se existe na tabela) para indicar
                #Que devemos descer para contexto menor
                tabela = self._criar_tabela_frequencia(probs)
                self.codificador.write(tabela, self.simbolo_para_int[_SIMBOLO_ESCAPE])

            # Exclui todos os símbolos visíveis neste nó para não contá-los
            # Novamente em contextos menores
            excluidos.update(c.simbolo for c in no_contexto.filhos.values() if c.simbolo is not _SIMBOLO_ESCAPE)
            # Segue para contexto menor via vine
            no_contexto = no_contexto.vine

        #Caso 3: contexto de ordem -1
        todos = set(self.simbolo_para_int.keys()) - {_SIMBOLO_ESCAPE}
        restantes = sorted(list(todos - excluidos), key=lambda s: self.simbolo_para_int[s])

        if restantes:
            #Probabilidade uniforme (equiprobabilidade) sobre símbolos restantes
            prob_simbolo = 1 / len(restantes)
            self.soma_entropia += -math.log2(prob_simbolo)
            self.contagem_simbolos += 1

        #Constrói tabela com freq 1 para cada símbolo restante (fallback básico)
        freqs = [0] * self.tamanho_alfabeto_total

        for s in restantes:
            freqs[self.simbolo_para_int[s]] = 1

        self.codificador.write(arithmeticcoding.SimpleFrequencyTable(freqs), simbolo_int)
        self._atualizar_modelo(simbolo)

    # -----------------------------------------------------
    # Decodificação de um símbolo (simétrica à codificação)
    # -----------------------------------------------------
    def decodificar_simbolo(self) -> Any:
        """
        Decodifica um símbolo a partir do fluxo de bits usando o modelo PPM
        atual e o decodificador aritmético.

        Procedimento:
        - Na ordem, tenta construir a tabela a partir do contexto atual
        - Usa o decodificador aritmético para ler um inteiro indexado pela tabela
        - Se o inteiro corresponder ao símbolo de escape, desce para contexto menor
        - Se corresponder a um símbolo real, atualiza entropia (simetricamente)
          e retorna o símbolo
        - Se nenhum contexto produz probabilidades (ordem -1), usa o fallback
          uniforme sobre símbolos restantes (mesma tabela construída durante a
          codificação)
        """
        excluidos = set()
        no_contexto = self.base

        while no_contexto is not None:
            probs = self._obter_probabilidades(no_contexto, excluidos)

            if probs:
                tabela = self._criar_tabela_frequencia(probs)
                decod_int = self.decodificador.read(tabela)
                simbolo = self.int_para_simbolo[decod_int]

                if simbolo is not _SIMBOLO_ESCAPE:
                    # Se é um símbolo real, atualiza entropia (mantendo simetria)
                    total = sum(probs.values())

                    if total > 0 and simbolo in probs and probs[simbolo] > 0:
                        prob_simbolo = probs[simbolo] / total
                        self.soma_entropia += -math.log2(prob_simbolo)
                        self.contagem_simbolos += 1

                    self._atualizar_modelo(simbolo)
                    return simbolo

            # Se descodificou ESCAPE ou não há probs, marca símbolos deste nó
            excluidos.update(c.simbolo for c in no_contexto.filhos.values() if c.simbolo is not _SIMBOLO_ESCAPE)
            no_contexto = no_contexto.vine

        # Fallback ordem -1
        todos = set(self.simbolo_para_int.keys()) - {_SIMBOLO_ESCAPE}
        restantes = sorted(list(todos - excluidos), key=lambda s: self.simbolo_para_int[s])

        if restantes:
            prob_simbolo = 1 / len(restantes)
            self.soma_entropia += -math.log2(prob_simbolo)
            self.contagem_simbolos += 1

        freqs = [0] * self.tamanho_alfabeto_total

        for s in restantes:
            freqs[self.simbolo_para_int[s]] = 1

        decod_int = self.decodificador.read(arithmeticcoding.SimpleFrequencyTable(freqs))
        simbolo = self.int_para_simbolo[decod_int]
        self._atualizar_modelo(simbolo)

        return simbolo

    # -----------------------------------------------
    # Finalização da compressão (escreve EOF e flush)
    # -----------------------------------------------
    def finalizar_compressao(self) -> bytes:
        """
        Escreve o símbolo de fim e finaliza o codificador aritmético, retornando
        o buffer de bytes resultante.

        Também escreve bits de padding (zeros) até que o último byte seja
        completado, conforme a interface do BitOutputStream usado.
        """
        self.codificar_simbolo(_SIMBOLO_FIM)
        self.codificador.finish()

        while self.escritor_bits.numbitsfilled != 0:
            self.escritor_bits.write(0)

        resultado = self.escritor_bits.output.getvalue()
        self.escritor_bits.output.close()

        return resultado

    # -----------------------------
    # Persistência do modelo (salvar/carregar)
    # -----------------------------
    def salvar_modelo(self, nome_arquivo: str):
        #Salva o objeto da árvore (self) em pickle para reutilização posterior
        with open(nome_arquivo, "wb") as f:
            pickle.dump(self, f)

    @staticmethod
    def carregar_modelo(nome_arquivo: str) -> "ArvorePPM_C":
        #Carrega um modelo salvo via pickle e retorna a instância.
        with open(nome_arquivo, "rb") as f:
            return pickle.load(f)

    # -----------------------------
    # Geração de texto (a partir do modelo treinado)
    # -----------------------------
    def gerar_texto(self, tamanho: int, semente: str = "") -> str:
        """
        Gera texto aleatório de comprimento `tamanho` usando as distribuições
        condicionais aprendidas pelo modelo. A semente (seed) é incorporada ao
        contexto inicial para condicionar a geração, caso desejado.

        Estratégia:
        - Percorre contextos do maior para o menor (via vine) procurando símbolos
          inteiros (bytes) visíveis e seleciona aleatoriamente segundo as
          probabilidades empíricas normalizadas, ou seja, considerando as probabilidades
          como pesos.
        - Se não encontra símbolo nos contextos, usa os símbolos disponíveis na raiz
          como fallback (evita falha quando a trie está escassa).
        """
        resultado: List[str] = []
        # Incorpora a semente ao contexto (cada caractere -> código inteiro)
        for ch in semente:
            simbolo_int = ord(ch)
            self._atualizar_modelo(simbolo_int)
            resultado.append(ch)

        for _ in range(tamanho):
            no_atual = self.base
            excluidos: Set[Any] = set()
            escolhido: Optional[int] = None

            while no_atual is not None:
                probs = self._obter_probabilidades(no_atual, excluidos)
                if probs:
                    # Usamos apenas símbolos inteiros para saída textual
                    probs_validas = {s: c for s, c in probs.items() if isinstance(s, int)}

                    if probs_validas:
                        simbolos, pesos = zip(*probs_validas.items())
                        total = sum(pesos)
                        pesos = [p / total for p in pesos]
                        escolhido = random.choices(simbolos, weights=pesos, k=1)[0]
                        break

                excluidos.update(c.simbolo for c in no_atual.filhos.values() if isinstance(c.simbolo, int))
                no_atual = no_atual.vine

            if escolhido is None:
                # Fallback para símbolos disponíveis na raiz (se houver)
                disponiveis = [s for s in self.raiz.filhos.keys() if isinstance(s, int)]
                escolhido = random.choice(disponiveis) if disponiveis else ord(' ')

            self._atualizar_modelo(escolhido)
            resultado.append(chr(escolhido))

        return ''.join(resultado)
    
    def gerar_texto_nitidez(self, tamanho: int, semente: str = "", nitidez: float = 1.0) -> str:
        """
        Gera texto aleatório com saídas de depuração detalhadas a cada passo.

        Para cada símbolo a ser gerado, esta função imprime:
        - O contexto atual a partir do qual a previsão é feita.
        - As probabilidades dos símbolos candidatos, mostrando a original e a ajustada.
        - O símbolo que foi efetivamente escolhido com base nessas probabilidades.

        Parâmetros:
        - tamanho (int): Quantidade de caracteres a serem gerados.
        - semente (str): Texto inicial para condicionar a geração.
        - nitidez (float): Fator para ajustar a distribuição de probabilidade.
            - nitidez > 1.0: Torna as escolhas mais prováveis ainda mais prováveis (menos aleatório).
            - nitidez = 1.0: Comportamento padrão, sem ajustes.
            - 0.0 < nitidez < 1.0: Torna as escolhas mais aleatórias, achatando a distribuição.
        """

        resultado: List[str] = []
        # Incorpora a semente ao contexto, se houver
        if semente:
            for ch in semente:
                simbolo_int = ord(ch)
                self._atualizar_modelo(simbolo_int)
                resultado.append(ch)

        for i in range(tamanho):
            contexto_str = ''.join(map(chr, self.contexto))

            no_atual = self.base
            excluidos: Set[Any] = set()
            escolhido: Optional[int] = None
            found_in_context = False

            while no_atual is not None:
                probs = self._obter_probabilidades(no_atual, excluidos)
                if probs:
                    # Filtra para manter apenas símbolos que são bytes/inteiros
                    probs_validas = {s: c for s, c in probs.items() if isinstance(s, int)}

                    if probs_validas:

                        # Ordena por contagem para melhor visualização
                        sorted_probs = sorted(probs_validas.items(), key=lambda item: -item[1])
                        simbolos = [item[0] for item in sorted_probs]
                        pesos_originais = [item[1] for item in sorted_probs]
                        
                        # Aplica o fator de nitidez para inflar/achatar as probabilidades
                        # Elevar a contagem a uma potência > 1 infla as maiores contagens
                        pesos_ajustados = [p ** nitidez for p in pesos_originais]

                        total_original = sum(pesos_originais)
                        total_ajustado = sum(pesos_ajustados)
                        
                        for idx, s in enumerate(simbolos):
                            p_original = pesos_originais[idx]
                            p_ajustado = pesos_ajustados[idx]
                            
                            prob_orig_percent = (p_original / total_original) * 100 if total_original > 0 else 0
                            prob_ajus_percent = (p_ajustado / total_ajustado) * 100 if total_ajustado > 0 else 0

                        # Realiza a escolha aleatória ponderada com os pesos ajustados
                        escolhido = random.choices(simbolos, weights=pesos_ajustados, k=1)[0]
                        
                        found_in_context = True
                        break

                excluidos.update(c.simbolo for c in no_atual.filhos.values() if isinstance(c.simbolo, int))
                no_atual = no_atual.vine

            if not found_in_context:
                disponiveis = [s for s in self.raiz.filhos.keys() if isinstance(s, int)]
                if disponiveis:
                    escolhido = random.choice(disponiveis)
                else:
                    escolhido = ord(' ')

            self._atualizar_modelo(escolhido)
            resultado.append(chr(escolhido))

        texto_gerado = ''.join(resultado)

        return texto_gerado
    
    def calcular_perplexidade(self) -> float:
        """
        Calcula a perplexidade com base na entropia acumulada.
        Deve ser chamada após o processamento de um texto (compressão/descompressão).
        Retorna float('inf') se nenhum símbolo foi processado.
        """
        if self.contagem_simbolos == 0:
            return float('inf')

        entropia_media = self.soma_entropia / self.contagem_simbolos
        perplexidade = math.pow(2, entropia_media)
        return perplexidade

# --------------------
# Funções utilitárias
# --------------------

def comprimir(entrada: str, saida: str, ordem: int):
    """
    Compacta o arquivo `entrada` para `saida` usando um modelo PPM de ordem
    `ordem`. Retorna (tempo_em_segundos, instancia_compressor).

    Passos:
    - Cria instância ArvorePPM_C para codificação
    - Lê o arquivo byte a byte e chama codificar_simbolo para cada byte
    - Finaliza a compressão escrevendo o buffer resultante em `saida`
    """
    print(f"Iniciando compressão de '{entrada}'...")

    inicio = time.perf_counter()
    compressor = ArvorePPM_C(ordem=ordem, tamanho_alfabeto=256)

    with open(entrada, "rb") as fin:
        while True:
            byte = fin.read(1)
            if not byte:
                break
            compressor.codificar_simbolo(byte[0])

    dados = compressor.finalizar_compressao()

    with open(saida, "wb") as fout:
        fout.write(dados)

    fim = time.perf_counter()
    print(f"Compressão finalizada em {fim - inicio:.2f} segundos.")

    return fim - inicio, compressor


def descomprimir(entrada: str, saida: str, ordem: int):
    """
    Descompacta `entrada` para `saida` usando um modelo PPM de ordem `ordem`.

    Passos:
    - Lê todo o arquivo comprimido em um buffer de bits
    - Inicializa ArvorePPM_C em modo decodificação (passando fluxo_bits)
    - Chama decodificar_simbolo em loop até encontrar _SIMBOLO_FIM
    - Grava bytes reconstruídos em `saida`
    """
    print(f"\nIniciando descompressão de '{entrada}'...")
    inicio = time.perf_counter()

    with open(entrada, "rb") as f:
        fluxo_bits = io.BytesIO(f.read())

    descompressor = ArvorePPM_C(ordem=ordem, tamanho_alfabeto=256, fluxo_bits=fluxo_bits)

    with open(saida, "wb") as fout:
        while True:
            simbolo = descompressor.decodificar_simbolo()
            if simbolo == _SIMBOLO_FIM:
                break
            # Quando simbolo é um inteiro (byte), grava no arquivo
            fout.write(bytes([simbolo]))

    fim = time.perf_counter()
    print(f"Descompressão finalizada em {fim - inicio:.2f} segundos.")

    return fim - inicio


def comparar_arquivos(arq1: str, arq2: str) -> bool:
    """
    Compara dois arquivos em modo binário retornando True se forem idênticos.
    Implementado para ser eficiente em memória usando buffer de tamanho fixo.
    """
    try:
        buffer = 8192
        with open(arq1, 'rb') as f1, open(arq2, 'rb') as f2:
            while True:
                b1 = f1.read(buffer)
                b2 = f2.read(buffer)
                if b1 != b2:
                    return False
                if not b1:
                    return True
    except FileNotFoundError:
        return False


def preprocessar_texto(texto: str) -> str:
    """
    Exemplo simples de pré-processamento de texto: normaliza para minúsculas e
    remove caracteres que não sejam letras a-z ou espaço.
    """
    texto = texto.lower()
    texto = re.sub(r'[^a-z., ]+', '', texto)
    texto = re.sub(r'[\s]+', ' ', texto)
    return texto

In [ ]:
# -----------------------------
# Bloco principal de execução (exemplo de uso)
# -----------------------------
if __name__ == "__main__":
    nome_entrada = "files_concat.txt"
    nome_comprimido = "files_concat.bin"
    nome_descomprimido = "files_concat_descomprimido.txt"

    # Ordem do modelo PPM. Ordens maiores capturam contextos mais longos
    # (melhor modelagem para texto natural) mas consomem mais memória e tempo.
    ORDEM_PPM = 4

    # -----------------------------
    # 1) Compressão
    # -----------------------------
    # Chamamos a função `comprimir` que lê o arquivo byte-a-byte, codifica
    # cada símbolo e retorna o tempo de compressão e a instância do
    # compressor (ArvorePPM_C) que contém estatísticas como soma_entropia.
    tempo_compressao, compressor = comprimir(nome_entrada, nome_comprimido, ORDEM_PPM)

    # -----------------------------
    # 2) Descompressão
    # -----------------------------
    # Chamamos `descomprimir` que reconstrói o arquivo a partir do binário
    # comprimido usando o mesmo modelo PPM (ordem) durante a decodificação.
    tempo_descompressao = descomprimir(nome_comprimido, nome_descomprimido, ORDEM_PPM)

    # -----------------------------
    # 3) Medidas e relatório
    # -----------------------------
    # Tamanhos em bytes dos arquivos original e comprimido
    tamanho_original = os.path.getsize(nome_entrada)
    tamanho_comprimido = os.path.getsize(nome_comprimido)

    # Comprimento médio em bytes por símbolo (aqui usamos 1 símbolo = 1 byte)
    # Se o arquivo original tiver tamanho 0, evitamos divisão por zero
    comprimento_medio_bytes_por_simbolo = (tamanho_comprimido) / tamanho_original if tamanho_original > 0 else 0

    # Entropia empírica estimada pelo modelo durante a compressão
    # Usamos os campos acumulados no objeto compressor: soma_entropia e contagem_simbolos
    entropia_fonte = compressor.soma_entropia / compressor.contagem_simbolos if compressor.contagem_simbolos > 0 else 0

    # Convertemos comprimento médio para bits/símbolo multiplicando por 8
    comprimento_medio_bits_por_simbolo = comprimento_medio_bytes_por_simbolo * 8

    # Cálculo da perplexidade para o arquivo comprimido
    perplexidade = compressor.calcular_perplexidade()

    # - Tempo de compressão/descompressão: em segundos
    # - Tamanhos: em bytes
    # - Entropia da fonte: em bits por símbolo
    # - Comprimento médio: bits por símbolo
    print("\n" + "="*25 + " RELATÓRIO FINAL " + "="*25)
    print(f"Tempo de Compressão.....: {tempo_compressao:.2f} segundos")
    print(f"Tempo de Descompressão..: {tempo_descompressao:.2f} segundos")
    print(f"Tamanho Original........: {tamanho_original} Bytes")
    print(f"Tamanho Comprimido......: {tamanho_comprimido} Bytes")
    print(f"Entropia da Fonte (H(X)): {entropia_fonte:.4f} bits/símbolo")
    print(f"Comprimento Médio.......: {comprimento_medio_bits_por_simbolo:.4f} bits/símbolo")
    print(f"Perplexidade do modelo..: {perplexidade:.4f}")
    print("-" * 67)

    # -----------------------------
    # 4) Verificação de integridade
    # -----------------------------
    # Comparamos byte a byte o arquivo original com o descomprimido. Se
    # forem idênticos, a compressão/descompressão foi lossless.
    sao_iguais = comparar_arquivos(nome_entrada, nome_descomprimido)

    if sao_iguais:
        print("Verificação: O arquivo original e o descomprimido são IDÊNTICOS.")
    else:
        print("Verificação: FALHA! O arquivo original e o descomprimido são DIFERENTES.")
    print("="*67)

In [ ]:
# Tenta carregar o modelo existente
nome_modelo = "ppm_gutenberg_k_4_pont.pkl"
if os.path.exists(nome_modelo):
    print(f"Carregando modelo PPM do arquivo: {nome_modelo}")
    modelo = ArvorePPM_C.carregar_modelo(nome_modelo)

    # Correção para garantir que os objetos especiais tenham a mesma referência
    # após o carregamento do modelo.
    modelo.simbolo_para_int[_SIMBOLO_ESCAPE] = modelo.tamanho_alfabeto_bytes
    modelo.simbolo_para_int[_SIMBOLO_FIM] = modelo.tamanho_alfabeto_bytes + 1
    modelo.int_para_simbolo[modelo.tamanho_alfabeto_bytes] = _SIMBOLO_ESCAPE
    modelo.int_para_simbolo[modelo.tamanho_alfabeto_bytes + 1] = _SIMBOLO_FIM
    # --- FIM DAS LINHAS A SEREM INSERIDAS ---

else:
    print(f"Arquivo de modelo não encontrado. Criando um novo modelo.")
    modelo = ArvorePPM_C(ordem=4, tamanho_alfabeto=256)

# Restante do código
nova_entrada = "medium_concat.txt"

if os.path.exists(nova_entrada):
    with open(nova_entrada, "r", encoding="utf-8") as f:
        texto_adicional = preprocessar_texto(f.read())

    for ch in texto_adicional:
        modelo.codificar_simbolo(ord(ch))

    # O símbolo de fim não é codificado aqui, pois o treinamento continuará.
    # Ele será codificado apenas na finalização da compressão.

    modelo.salvar_modelo(nome_modelo)
    print(f"Modelo atualizado e salvo em: {nome_modelo}")

else:
    print(f"Erro: O arquivo de nova entrada '{nova_entrada}' não foi encontrado.")

In [ ]:
def quebrar_linhas_por_caracter(texto: str, max_caracteres: int = 80) -> str:
    # Quebra o texto a cada `max_caracteres` caracteres (quebra fixa).
    if max_caracteres <= 0:
        return texto
    linhas = [texto[i:i + max_caracteres] for i in range(0, len(texto), max_caracteres)]
    return "\n".join(linhas)

def quebrar_linhas_por_palavras(texto: str, palavras_por_linha: int = 15) -> str:
    # Quebra o texto em linhas com até `palavras_por_linha`.
    if palavras_por_linha <= 0:
        return texto
    palavras = texto.split()
    linhas = [" ".join(palavras[i:i + palavras_por_linha]) for i in range(0, len(palavras), palavras_por_linha)]
    return "\n".join(linhas)

In [ ]:
# Carrega o modelo PPM previamente salvo (formato pickle)
modelo = ArvorePPM_C.carregar_modelo("ppm_gutenberg_1_100mb_k_4_pont.pkl")

In [ ]:
# Gera o número passado por parâmetro de símbolos/caracteres a partir do modelo
texto_gerado = modelo.gerar_texto(tamanho=400, semente='')

# Formata o texto quebrando por palavras (padrão: 15 palavras por linha)
texto_formatado = quebrar_linhas_por_palavras(texto_gerado)

print(texto_formatado)

In [ ]:
# Gera o número passado por parâmetro de símbolos/caracteres a partir do modelo
texto_gerado = modelo.gerar_texto_nitidez(tamanho=400, semente='', nitidez=2.0)

# Formata o texto quebrando por palavras (padrão: 15 palavras por linha)
texto_formatado = quebrar_linhas_por_palavras(texto_gerado)

print(texto_formatado)

### Métrica Distinct-n

In [ ]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt') # Baixa o tokenizador, se necessário

def calcular_distinct_n(texto: str, n: int) -> float:
    """
    Calcula a métrica Distinct-n para um dado texto.

    Args:
    - texto: O texto gerado pelo modelo.
    - n: A ordem do n-grama (1 para unigrama, 2 para bigrama, etc.).

    Returns:
    - A proporção de n-gramas únicos.
    """
    if not texto.strip():
        return 0.0

    tokens = word_tokenize(texto.lower())
    if len(tokens) < n:
        return 0.0

    n_grams = list(ngrams(tokens, n))
    if not n_grams:
        return 0.0

    return len(set(n_grams)) / len(n_grams)

In [ ]:
# Gerar texto
texto_gerado = modelo.gerar_texto_nitidez(tamanho=400, semente="the king", nitidez=2.0)

# Calcular as métricas de diversidade
distinct_1 = calcular_distinct_n(texto_gerado, 1)
distinct_2 = calcular_distinct_n(texto_gerado, 2)

print(f"Texto Gerado: '{texto_gerado}...'")
print(f"Distinct-1: {distinct_1:.4f}")
print(f"Distinct-2: {distinct_2:.4f}")

In [ ]:
texto_gerado = modelo.gerar_texto(tamanho=400, semente="the king")

# Calcular as métricas de diversidade
distinct_1 = calcular_distinct_n(texto_gerado, 1)
distinct_2 = calcular_distinct_n(texto_gerado, 2)

print(f"Texto Gerado: '{texto_gerado}...'")
print(f"Distinct-1: {distinct_1:.4f}")
print(f"Distinct-2: {distinct_2:.4f}")

### Testes

In [ ]:
# Carrega o modelo PPM previamente salvo (formato pickle)
modelo = ArvorePPM_C.carregar_modelo("ppm_gutenberg_k_4_pont.pkl")

In [ ]:
texto_gerado = modelo.gerar_texto(tamanho=400, semente="the king")
texto_formatado = quebrar_linhas_por_palavras(texto_gerado)

print(texto_formatado)

In [ ]:
texto_gerado = modelo.gerar_texto_nitidez(tamanho=400, semente="the king", nitidez=2.0)
texto_formatado = quebrar_linhas_por_palavras(texto_gerado)

print(texto_formatado)